In [4]:
import numpy as np
from collections import defaultdict

In [103]:
ipa_seqs = [
    [1, 10, 5],
    [10, 25, 9],
]

whisper_seqs = [
    [20, 9, 2],
    [11, 13, 6, 10],
]

ipa_vocab_size = np.max(np.concatenate(ipa_seqs))+1
whisper_vocab_size = np.max(np.concatenate(whisper_seqs))+1

eps=1e-6
sum_sqd_dist = np.full([whisper_vocab_size, ipa_vocab_size], eps)
ipa_token_counts = np.full(ipa_vocab_size, eps)

In [104]:
sum_sqd_dist.shape

(21, 26)

In [105]:
sum_sqd_dist.sum(axis=0).shape

(26,)

In [107]:
def initialize_probs(ipa_seqs, whisper_seqs, sum_sqd_dist, ipa_token_counts):
    for ipa_seq, whisper_seq in zip(ipa_seqs, whisper_seqs):
        for i, ipa_token in enumerate(ipa_seq):
            ipa_token_counts[ipa_token]+=1
            for j, whisper_token in enumerate(whisper_seq):
                sum_sqd_dist[whisper_token,ipa_token]+=(i-j)**2
    prob_unnorm = sum_sqd_dist/(ipa_token_counts)
    sum_prob_unnorm = prob_unnorm.sum(axis=0)
    prob_norm=prob_unnorm/sum_prob_unnorm
    return prob_norm
    
initialize_probs(ipa_seqs, whisper_seqs, sum_sqd_dist, ipa_token_counts).T[1]

array([9.9999790e-08, 9.9999790e-08, 7.9999842e-01, 9.9999790e-08,
       9.9999790e-08, 9.9999790e-08, 9.9999790e-08, 9.9999790e-08,
       9.9999790e-08, 1.9999968e-01, 9.9999790e-08, 9.9999790e-08,
       9.9999790e-08, 9.9999790e-08, 9.9999790e-08, 9.9999790e-08,
       9.9999790e-08, 9.9999790e-08, 9.9999790e-08, 9.9999790e-08,
       9.9999790e-08])